In [1]:
# https://github.com/unitedstates/congress-legislators
# let's start with a list of current legislators

import requests
r = requests.get('https://theunitedstates.io/congress-legislators/legislators-current.json')

In [16]:
import json
congress_json = json.loads(r.text)

In [17]:
# define a function to let us download files
# https://stackoverflow.com/questions/16694907/how-to-download-large-file-in-python-with-requests-py
def download_file(url,downloadto=''):
    local_filename = downloadto + url.split('/')[-1]
    # NOTE the stream=True parameter
    r = requests.get(url, stream=True)
    with open(local_filename, 'wb') as f:
        for chunk in r.iter_content(chunk_size=1024): 
            if chunk: # filter out keep-alive new chunks
                f.write(chunk)
    return local_filename

In [18]:
# let's download the relevant photos
# https://github.com/unitedstates/images

for congressman in congress_json:
    bioguide = congressman['id']['bioguide']
    photo_url = r'https://theunitedstates.io/images/congress/225x275/'+bioguide+'.jpg'
    download_file(photo_url,downloadto=r'images/')

In [38]:
# now let's pull a few different datasets for the folks we have

propublica_api_key = open(r"propublica_api_key.txt","r").read()
propublica_api_key

def get_recent_votes():

    # put your key in the header
    headers = {
        "X-API-Key": propublica_api_key
    }

    url = "https://api.propublica.org/congress/v1/house/votes/recent.json"

    # make request
    r = requests.get(url, headers=headers)

    return(json.loads(r.text))


def get_vote_breakdown(vote_number):

    # put your key in the header
    headers = {
        "X-API-Key": propublica_api_key
    }

    url = r"https://api.propublica.org/congress/v1/115/senate/sessions/1/votes/"+vote_number+r".json"

    # make request
    r = requests.get(url, headers=headers)

    return(json.loads(r.text))


    
recent_votes = get_recent_votes()

In [37]:
for vote in recent_votes['results']['votes']:
    roll_call_number = vote['roll_call']
    print(vote['bill']['title'])
    print()

166
None

165
To reauthorize programs of the Federal Aviation Administration, and for other purposes.

164
To reauthorize programs of the Federal Aviation Administration, and for other purposes.

163
To reauthorize programs of the Federal Aviation Administration, and for other purposes.

162
To reauthorize programs of the Federal Aviation Administration, and for other purposes.

161
To impose additional sanctions with respect to serious human rights abuses of the Government of Iran, and for other purposes.

160
To reauthorize programs of the Federal Aviation Administration, and for other purposes.

159
To reauthorize programs of the Federal Aviation Administration, and for other purposes.

158
To reauthorize programs of the Federal Aviation Administration, and for other purposes.

157
To reauthorize programs of the Federal Aviation Administration, and for other purposes.

156
To reauthorize programs of the Federal Aviation Administration, and for other purposes.

155
To reauthorize pro

In [41]:
vote_breakdown = get_vote_breakdown("147")

In [94]:
import pandas as pd


def get_vote_positions_dataframe(vote_breakdown):
    
    roll_call_number = vote_breakdown['results']['votes']['vote']['roll_call']
    vote_positions = vote_breakdown['results']['votes']['vote']['positions']

    temp_df = pd.DataFrame()
    
    for position in vote_positions:
        row = {
            'bioguide': position['member_id'],
            str(roll_call_number): position['vote_position']
        }
        temp_df = temp_df.append(row,ignore_index=True)
        
    return temp_df



In [95]:
roll_call_numbers_list = range(1,101)


def create_voting_dataframe(roll_call_numbers_list):
    
    votes_df = pd.DataFrame()

    for roll_call_number in roll_call_numbers_list:

        roll_call_string = str(roll_call_number)
        vote_breakdown = get_vote_breakdown(roll_call_string)

        try:
            votes_df = pd.merge(votes_df,get_vote_positions_dataframe(vote_breakdown),on='bioguide')

        except:
            votes_df = get_vote_positions_dataframe(vote_breakdown)
        
    return votes_df
        
votes_df = create_voting_dataframe(roll_call_numbers_list)

In [97]:
votes_df['yeses'] = 0
votes_df['nos'] = 0
votes_df['abstentions'] = 0

for vote in roll_call_numbers_list:
    votes_df.loc[votes_df[str(vote)] == 'Yes','yeses'] = votes_df.loc[votes_df[str(vote)] == 'Yes','yeses'] + 1
    votes_df.loc[votes_df[str(vote)] == 'No','nos'] = votes_df.loc[votes_df[str(vote)] == 'No','nos'] + 1
    votes_df.loc[votes_df[str(vote)] == 'Not Voting','abstentions'] = votes_df.loc[votes_df[str(vote)] == 'Not Voting','abstentions'] + 1
        
votes_summary = votes_df[['bioguide','yeses','nos','abstentions']]
votes_summary

,bioguide,yeses,nos,abstentions
0,A000360,79,18,3
1,B001230,29,71,0
2,B001261,81,18,1
3,B001267,36,64,0
4,B001277,30,69,1
5,B000575,82,16,2
6,B001288,24,76,0
7,B001236,83,17,0
8,B000944,36,64,0
9,B001135,83,17,0


In [100]:
votes_summary['imageurl'] = r'images/' + votes_summary['bioguide'] + '.jpg'
votes_summary.to_csv('data/congressdata.csv')

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
